# Universidad Politécnica Salesiana 
## Presentación de modelo de red neuronal convolucional CNN

Descripción Dataset: 
 
En la presente practica realizaremos la clasificación entre perros y gatos en base al conjunto de datos llamado Asirra disponible en kaggle consiste en 25,000 de imágenes que fueron clasificadas manualmente por personas de miles de refugios de animales dichas imágenes están etiquetadas como 1= perro, 0= gato , este conjunto de datos tiene cierto grado de complejidad porque las imágenes ofrecen distintos tonos de iluminación y distintas poses en las que se encuentran los perros como los gatos.

Enlace 
[https://www.kaggle.com/c/dogs-vs-cats](https://)

<h1>Se importan las librerias necesarias para la practica</h1>

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import os
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from keras.models import model_from_json

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", "r") as z:
    z.extractall(".")

with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", "r") as z:
    z.extractall(".")


<h2>Descripción del Dataset</h>


Este conjunto de datos está conformado por 25.000 imágenes entonces en el siguiente bloque de código procederemos a obtener información de las etiquetas, recorriéndolas mediante un ciclo for, Luego procedemos a generar el dataframe para su posterior estudio.

In [ ]:
# Lista de las categorias
categorias = []
# Lista de archivos dentro de los datos de train
archivos = os.listdir("train")
# Bucle para obtener los nombres de los archivos (Etiquetas de salida)
for archivo in archivos:
    categoria = archivo.split('.')[0]
    if categoria == 'dog':
        categorias.append("dog")
    else:
        categorias.append("cat")
        
# Se crea un dataframe en base del nombre del archivo y la categoria (tipo de animal)
df = pd.DataFrame({
    'archivo': archivos,
    'categoria': categorias
})
print(categoria, '\n', df)

<h2>Splitting Del Dataset</h2>

 Separamos el conjunto de train y test, luego indicamos que los índices o puteros estén en el comienzo de los dataframes.

In [ ]:
# Separamos dos conjuntos, train(80% del dataset) y test(20%).
train, test = train_test_split(df, test_size=0.20, random_state=42)

# Reseteamos los index para vayan al comienzo de daatframe.
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# Imprimimos el tamano de los dataframes.
print(train.shape, '<-->', test.shape)

<h2>Preprocesamiento De Los Datos</h2>

Establecemos un conjunto de parámetros para generar más imágenes partiendo de las que ya tenemos, buscando generar más datos para que la red neuronal entrene con mas información, estos parámetros se fijan en la variable train_image_data. Luego hacemos uso del método flow_from_dataframe() donde especificamos los parámetros anteriores y hacemos referencia a el apartado de train del dataset anterior, pro ultimo procedemos a realizar una normalización de los valores de los pixeles. 

Las imágenes de este conjunto de datos tienen la dimensión de 128x128 y contiene tres canales.

In [ ]:
# Generador De Entrenamiento (Preprocesamiento de las imagenes)
TAM_IMG = (128, 128)
"""
Generamos nuevas imagenes en base a las pre existentes, por ejmplo, podemos cambiar el rango de rotacion, 
cortamos la imagen, hacemos zoom, volteamos, entre otras tecnicas para que de esta manera podamos tener
una mayor cantidad de imagenes para entrenar y hacer el testing.

"""
train_image_data = ImageDataGenerator(
    rotation_range = 15,
    rescale = 1./255,
    shear_range = 0.1,
    zoom_range=0.2,
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

# Procedemos ha generar las imagenes y preprocesarlas.
train_generador = train_image_data.flow_from_dataframe(
    train,
    'train/',
    x_col = 'archivo',
    y_col = 'categoria',
    target_size = TAM_IMG,
    class_mode = 'categorical',
    batch_size = 32
)

# Escalamos la imagen diviendo cada pixel para 255.
test_image_data = ImageDataGenerator(rescale=1./255)
test_generador = test_image_data.flow_from_dataframe(
    test,
    'train/',
    x_col = 'archivo',
    y_col = 'categoria',
    target_size = TAM_IMG,
    class_mode = 'categorical',
    batch_size = 32
)

<h2>Modelo De La Red Neuronal Convolucional</h2>

El modelo de red neuronal propuesto presentamos tres de capas convolucionales de 32, 64, 128 neuronas las donde se aplica un filtro(kernel) de 3x3 con activación relu (en la primera capa se establece la dimensión de la imagen en este caso es 128x128 con 3 canales), además se realizamos un max pooling (obtiene los pixeles más representativos) de una dimensión de 2x2, La capa de aplanamiento consta de 256 neuronas.

En la capa de salida tenemos solamente dos neuronas porque en este caso de estudio vamos a clasificar dos clases (perros y gatos) donde aplicamos una activación softmax.

El modelo general tendrá una función de perdida categorical_crossentropy en conjunto con un optimizador rmsprop.


In [ ]:
# Modelo para el entrenamiento
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

modelo = Sequential()

modelo.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(64, (3, 3), activation='relu'))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(128, (3, 3), activation='relu'))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(128, (3, 3), activation='relu'))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.25))

modelo.add(Flatten())
modelo.add(Dense(512, activation = 'relu'))
modelo.add(BatchNormalization())
modelo.add(Dropout(0.5))
modelo.add(Dense(2, activation='softmax'))

modelo.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
modelo.summary()

<h2> Entrenamiento De La CNN </h2>

In [ ]:
train_shape = train.shape[0]
test_shape = test.shape[0]

history = modelo.fit_generator(
    train_generador,
    epochs = 25,
    validation_data = test_generador,
    validation_steps = test_shape//64,
    steps_per_epoch = train_shape//64,
)

<h2>Obtención De Los Resultados </h2>

In [ ]:
accuracy_training = history.history['accuracy']
accuracy_testing = history.history['val_accuracy']
epochs = 26

In [ ]:
plt.rcParams["figure.figsize"] = (12,8)
plt.grid()
plt.plot(accuracy_training, color='b', label="Training accuracy")
plt.xticks(np.arange(1, epochs, 1))
plt.xlabel('EPOCHS')
plt.ylabel('Accuracy en Training')
plt.tight_layout()
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = (12,8)
plt.grid()
plt.plot(accuracy_testing, color='r')
plt.xticks(np.arange(1, epochs, 1))
plt.xlabel('EPOCHS')
plt.ylabel('Accuracy en Testing')
plt.tight_layout()
plt.show()


<h2> Almacenamiento Del Modelo y Pesos </h2>

Declaramos las funciones que nos permitirán guardar y cargar los modelos de la red neuronal y también guardar los pesos de la misma.

In [ ]:
def guardar_pesos(nombre_archivo='pesos.h5'):
    modelo.save_weights(nombre_archivo)

def guardar_modelo(nombre_archivo='modelo.json'):
    modelo_json = modelo.to_json()
    with open(nombre_archivo, "w") as json_file:
        json_file.write(modelo_json)

def cargar_modelo(archivo_modelo='modelo.json', archivo_pesos='pesos.h5'):
    with open(archivo_modelo, 'r') as f:
        modelo = model_from_json(f.read())
    
    modelo.load_weights(archivo_pesos)
    
    return modelo

modelo = cargar_modelo(archivo_modelo='../input/cats-v-dogs-leo/modelo.json', archivo_pesos='../input/cats-v-dogs-leo/pesos.h5')

<h2> Listado De Archivos Para Testing </h2>

In [ ]:
archivos_test = os.listdir("test1")
test1 = pd.DataFrame({
    'archivo': archivos_test
})
muestras = test1.shape[0]
print(muestras)

<h2> Preprocesamiento De Los Datos Para Testing </h2>

Declaramos un Preprocesador para las imagenes de test, este procesador se encarga de genenrar mas imágenes para que la red aprenda con un numero mayor de registros en test.

In [ ]:
# Preprocesamiento y generador de imagenes para testing.
TAM_IMG = (128, 128)

test_generador = ImageDataGenerator(rescale=1./255)
test_gen = test_generador.flow_from_dataframe(
    test1,
    'test1',
    x_col = 'archivo',
    y_col = None,
    class_mode = None,
    target_size = TAM_IMG,
    batch_size = 32,
    shuffle = False
)

<h2> Predicción De Los Datos De Testing </h2>

In [ ]:
prediccion = modelo.predict_generator(test_gen, steps=np.ceil(muestras/32))

<h2> Agrupamiento De Los Resultados</h2>

In [ ]:
test1['categoria'] = np.argmax(prediccion, axis = 1)
label_map = dict((v, k) for k, v in train_generador.class_indices.items())
test1['categoria'] = test1['categoria'].replace(label_map)

<h2> Predicción De Resultados A Través De URL's </h2>

Declaramos una función que nos permite predecir si lo que representa una imagen corresponde a un perro o gato, como parámetro recibimos la url de la imagen a clasificar y como resultado obtenemos imprimimos la imagen ingresada acompañada del porcentaje de precisión con la que la red neuronal determina para esa imagen. 

In [ ]:
from io import BytesIO
from six.moves import urllib
from keras.preprocessing import image

TAM_IMG = (128, 128)


def predecir_img(URL):
    img = None
    with urllib.request.urlopen(URL) as url:
        img = load_img(BytesIO(url.read()), target_size=(128, 128))
    imagen = img
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255
    resultado = modelo.predict(img)
    val0 = resultado.tolist()[0][0]
    val1 = resultado.tolist()[0][1]
    if val0 > val1:
        return ('GATO', val0, imagen)
    elif val1 > val0:
        return ('PERRO', val1, imagen)

plt.figure(figsize=(12, 12))
    
animal, accuracy, imagen = predecir_img('https://static.iris.net.co/semana/upload/images/2019/6/18/620159_1.jpg')
print(animal +', probabilidad: ', str(round(accuracy*100, 2))+'%')
plt.subplot(5, 3, 1)
plt.imshow(imagen)

animal, accuracy, imagen = predecir_img('https://www.petdarling.com/articulos/wp-content/uploads/2014/08/gatos-persa.jpg?width=1200&enable=upscale')
print(animal +', probabilidad: ', str(round(accuracy*100, 2))+'%')
plt.subplot(5, 3, 2)
plt.imshow(imagen)

plt.tight_layout()
plt.show()

<h3> Predicción De Imágenes A Través De Imágenes De Testing </h3>

Método donde se puede visualizar una imagen del dataset determinando por un índice.

In [ ]:
# METODO 
from keras.preprocessing import image

def display_stats(sample_id = 5):
    train_len = len([name for name in os.listdir('test1')])
    
    if sample_id < train_len:
        names = []
        datos = dict()
        for name in os.listdir('test1'):
            names.append(name)
        file = load_img('test1/'+ names[sample_id], target_size=(128, 128))
        datos['Imagen:'] = names[sample_id]
        img = image.img_to_array(file)
        datos['Valor minimo:'] = np.min(img)
        datos['Valor maximo:'] = np.max(img)
        datos['Shape:'] = img.shape
        img = np.expand_dims(img, axis=0)
        img /= 255
        
        resultado = modelo.predict(img)
        val0 = resultado.tolist()[0][0]
        val1 = resultado.tolist()[0][1]
        if val0 > val1:
            #return ('GATO', val0, imagen)
            datos['% Prediccion:'] = round(val0*100, 2)
            datos['ETIQUETA'] = 'GATO'
        elif val1 > val0:
            datos['% Prediccion:'] = round(val1*100, 2)
            datos['ETIQUETA'] = 'PERRO'
        for x, y in datos.items():
            print(x, '\t', y)
        print()
        plt.imshow(file)
 
# EJEMPLO PERRO
display_stats(27)
# EJEMPLO GATO
#display_stats(98)


# Concluciones:

*  En cuanto a lo elaborado en esta práctica aprendemos sobre el manejo de imágenes para ingresar dicha información en una red neuronal aplicando reducción de dimensiones y capas que obtienen la información más relevante de las imágenes.
*  Frente a los resultados obtenidos podemos establecer que en determinados casos una optimización de parámetros no refleja mejora en la precisión de una red neuronal.
* En particular las redes neuronales tienen mejor desempeño en comparación con otros algoritmos ya que simulan el comportamiento de las redes neuronales humanas.
